# Exercises

There are three exercises in this notebook:

1. Use the cross-validation method to test the linear regression with different $\alpha$ values, at least three.
2. Implement a SGD method that will train the Lasso regression for 10 epochs.
3. Extend the Fisher's classifier to work with two features. Use the class as the $y$.

## 1. Cross-validation linear regression

You need to change the variable ``alpha`` to be a list of alphas. Next do a loop and finally compare the results.

In [1]:
import numpy as np
from sklearn.model_selection import KFold

In [2]:
x = np.array([188, 181, 197, 168, 167, 187, 178, 194, 140, 176, 168, 192, 173, 142, 176]).reshape(-1, 1)
y = np.array([141, 106, 149, 59, 79, 136, 65, 136, 52, 87, 115, 140, 82, 69, 121]).reshape(-1, 1)

x = np.asmatrix(np.c_[np.ones((15,1)), x])

I = np.identity(2)
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

results = []
for alpha in alphas:
    mse_list = []
    for train_idx, test_idx in kf.split(x):
        x_train, x_test = x[train_idx], x[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        w = np.linalg.inv(x_train.T * x_train + alpha * I) * x_train.T * y_train
        y_pred = x_test * w
        mse_list.append( float(np.mean((y_test.ravel() - y_pred.ravel().A1)**2)) )

    results.append(float(np.mean(mse_list)))

print('Results:')
for alpha, result in zip(alphas, results):
    print(f'Alpha: {alpha} \t Result: {result}')
print()
best_result = min(results)
best_alpha = alphas[results.index(best_result)]
print(f'Best alpha: {best_alpha}')

Results:
Alpha: 0.001 	 Result: 728.1805422778291
Alpha: 0.01 	 Result: 618.4111729996044
Alpha: 0.1 	 Result: 590.6667164595267
Alpha: 1 	 Result: 786.3752986524403
Alpha: 10 	 Result: 833.8706761853391
Alpha: 100 	 Result: 839.2205964358989

Best alpha: 0.1


## 2. Implement based on the Ridge regression example, the Lasso regression.

Please implement the SGD method and compare the results with the sklearn Lasso regression results. 

In [3]:
def predict(X, w, b):
    return (X.dot(w) + b)

In [4]:
def upadte_weights(X, Y, m, n, w, b, alpha, learning_rate):
    Y_prediction = predict(X, w, b)
    dw = np.zeros(n)
    for i in range(n):
      if w[i]>0:
        dw[i] = (-(2*(X[:,i]).dot(Y - Y_prediction)) + alpha) / m 
      else :
        dw[i] = (-(2*(X[:,i]).dot(Y - Y_prediction)) - alpha) / m

    db = - 2 * np.sum(Y - Y_prediction) / m

    w = w - learning_rate*dw
    b = b - learning_rate*db

    return w, b

In [5]:
def fit(X, Y, epochs, alpha, learning_rate):
    m = 15
    n = 1
    w = np.zeros(n)
    b = 0
    for i in range(epochs):
        w, b = upadte_weights(X, Y, m, n, w, b, alpha, learning_rate)
    return w, b

In [6]:
x = np.array([188, 181, 197, 168, 167, 187, 178, 194, 140, 176, 168, 192, 173, 142, 176]).reshape(-1, 1)
y = np.array([141, 106, 149, 59, 79, 136, 65, 136, 52, 87, 115, 140, 82, 69, 121])

I = np.identity(2)
alpha = 0.1 
w, b = fit(x, y, 10, 0.1, 0.00001)
print("SGD: ",w,b)
y = np.array([141, 106, 149, 59, 79, 136, 65, 136, 52, 87, 115, 140, 82, 69, 121]).reshape(-1, 1)
x = np.asmatrix(np.c_[np.ones((15,1)),x])
w = np.linalg.inv(x.T*x + alpha * I)*x.T*y # update this line
w=w.ravel()
print("Ridge: ",w)

from sklearn.linear_model import Lasso
x = np.array([188, 181, 197, 168, 167, 187, 178, 194, 140, 176, 168, 192, 173, 142, 176]).reshape(-1, 1)
y = np.array([141, 106, 149, 59, 79, 136, 65, 136, 52, 87, 115, 140, 82, 69, 121])
lasso_regression = Lasso(alpha=alpha)
lasso_regression.fit(X=x, y=y)
print("Lasso: ",lasso_regression.coef_, lasso_regression.intercept_ )

SGD:  [0.59386815] 0.0030520529904405663
Ridge:  [[-101.72397081    1.16978757]]
Lasso:  [1.61776499] -180.85790859980537


## 3. Extend the Fisher's classifier

Please extend the targets of the ``iris_data`` variable and use it as the $y$.

In [7]:
def calculate_class_means(x, y):
    classes = [0, 1, 2]
    class_means = []
    for class_number in classes:
        class_means.append(np.mean(x[y == class_number], axis=0).reshape(2, 1))
    return class_means

In [8]:
def calculate_within_class_scatter(class_means, x, y):
    classes = [0, 1, 2]
    SWs = []
    for class_number in classes:
        class_data = x[y == class_number]
        mean = class_means[class_number]
        temp = class_data.T - mean
        SW = temp @ temp.T
        SWs.append(SW)

    SW = np.zeros((2, 2))
    for _ in SWs:
        SW += _

    return SW

In [9]:
def calculate_between_class_scatter(x, class_means):
    classes = [0, 1, 2]
    global_mean = np.mean(x, axis=0).reshape(2, 1)
    SB = np.zeros((2, 2))
    for class_number in classes:
        mean_diff = class_means[class_number] - global_mean
        SB += 50 * (mean_diff @ mean_diff.T)
    return SB

In [10]:
def find_weights(SW, SB):
    eigenvals, eigenvecs = np.linalg.eig(np.linalg.inv(SW) @ SB)
    w = eigenvecs[:, np.argmax(eigenvals)].real
    return w

In [11]:
def get_boundaries(x, y, w):
    z = x @ w
    projected_means = [np.mean(z[y == k]) for k in [0, 1, 2]]
    boundaries = [(projected_means[0] + projected_means[1]) / 2, (projected_means[1] + projected_means[2]) / 2]
    return boundaries

In [12]:
def predict(sepal_length, sepal_width, w, boundaries):
    z_new = w[0] * sepal_length + w[1] * sepal_width
    if z_new < boundaries[0]:
        return 0
    elif z_new < boundaries[1]:
        return 1
    else:
        return 2

In [13]:
from sklearn.datasets import load_iris
import pandas as pd
iris_data = load_iris()
iris_df = pd.DataFrame(iris_data.data,columns=iris_data.feature_names)
x = iris_df.values # change here
x = iris_df[['sepal width (cm)', 'sepal length (cm)']].values
X = iris_df[['sepal width (cm)', 'sepal length (cm)']].values
y = iris_data.target
class_means = calculate_class_means(x, y)
SW = calculate_within_class_scatter(class_means, x, y)
SB = calculate_between_class_scatter(x, class_means)
w = find_weights(SW, SB)
boundaries = get_boundaries(x, y, w)